#### New image for testing- loading model weights

In [7]:
import csv
import math
import glob
import cv2
import numpy as np
from PIL import Image
import numpy as np
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape,GlobalAveragePooling2D,Dense
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon

In [8]:
WEIGHTS_FILE = "model_training_0.5_1.h5"
IMAGES = "./Test"
IMAGE_SIZE=96
ALPHA = 0.5


#### model creation

In [9]:
def create_model(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = trainable

    out = model.layers[-1].output

    x = Conv2D(4, kernel_size=3)(out)
    x = Reshape((4,), name="coords")(x)

    y = GlobalAveragePooling2D()(out)
    y = Dense(2, name="classes", activation="softmax")(y)

    return Model(inputs=model.input, outputs=[x, y])

In [10]:
model = create_model()
model.load_weights(WEIGHTS_FILE)

#### prediction display

In [17]:
unscaled = cv2.imread("./Test/test_4.jpg")
image_height, image_width, _= unscaled.shape

image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

region, class_id = model.predict(x=np.array([feat_scaled]))
region =  region[0]
print(region,class_id)
x0 = int(region[0] * image_width / IMAGE_SIZE)
y0 = int(region[1]  * image_height / IMAGE_SIZE)

x1 = int((region[0] + region[2]) * image_width / IMAGE_SIZE) 
y1 = int((region[1] + region[3]) * image_height / IMAGE_SIZE) 

class_id = np.argmax(class_id, axis=1)+1

print("x0:",x0)
print("y0:",y0)
print("x1:",x1)
print("y1:",y1)
print("class:",class_id)
print(image_height,image_width) 

[13.0846405 10.799796  82.484314  70.468994 ] [[0.6725313 0.3274687]]
x0: 408
y0: 229
x1: 2986
y1: 1726
class: [1]
2040 3000


In [ ]:
cv2.rectangle(unscaled, (x0, y0), (x1, y1), (255, 0, 0), 2)
#cv2.rectangle(unscaled, (x0_, y0_), (x1_, y1_), (0, 225, 0), 2)
cv2.putText(unscaled, "class: {}".format(class_id[0]), (x0, y0), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
cv2.imshow("image", unscaled)
cv2.waitKey(0)
cv2.destroyAllWindows()
